In [9]:
import os
import json
from typing import Optional
import numpy as np
import pytorch_lightning as pl
import cv2
from torch.utils.data import Dataset, DataLoader
# import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2
from albumentations import Compose, Resize, Normalize

PATH_DATA = "/home/ubuntu/datasets/segment_car_plate/data/"

In [10]:
with open(os.path.join(PATH_DATA, "train_recognition.json")) as fp:
    obj = json.load(fp)

In [14]:
obj[0]["file"], obj[0]["text"]

('train/0.box.00.jpg', 'B060BB125')

In [17]:
fname2text = {os.path.basename(rec["file"]):rec["text"] for rec in obj}

In [18]:
len(obj) == len(fname2text)

True

In [6]:
class RecDataset(Dataset):
    def __init__(self, data_dir, config):
        super(RecDataset, self).__init__()
        self.data_dir = data_dir
        self.abc = abc
        self.transforms = transforms
        self.split = split
        self.config = config
        self.images = [
            os.path.join(self.data_path,f"{name}.{ext}") for name, ext in config
        ]
        self.masks = [
            os.path.join(self.data_path,f"{name}.box.{ext}") for name, ext in config
        ]
    def 
#     def 